In [11]:
import numpy as np
import os
import cv2
import moviepy.editor as mpy
import glob


In [12]:
background = cv2.imread('Malibu.jpg')
source = cv2.imread('sour.png')

In [13]:
background_height = background.shape[0]
background_width = background.shape[1]
ratio = 360/background_height
background = cv2.resize(background,(int(background_width*ratio),360))


In [14]:
def histogram(inp):
    
    hist=np.zeros([256,1,3], dtype=np.uint8)
    for i in range(3):
        for g in range(256):
            hist[g]=np.count_nonzero(inp[:,:,i]==g)
        
    return hist
#inspired by slides of course

In [15]:
def LUT(cdf_image,cdf_source):
    
    LUT = np.zeros(256)
    gj = 0
    for gi in range(256):
        while cdf_source[gj] < cdf_image[gi] and gj < 255:
            gj += 1
        LUT[gi] = gj 
    return LUT


In [16]:
images = sorted(glob.glob('cat/cat_*.png'),key=len)
images_list=[]

In [17]:
hist_source = histogram(source)
cdf_source= hist_source.cumsum()/hist_source.sum()
hists=np.empty(hist_source.shape)

for filename in images:
    image = cv2.imread(filename)
    
    hist_image = histogram(image)
    hists=hists+hist_image

hist_image=hists/3
cdf_image = hist_image.cumsum()/hist_image.sum()
print(cdf_image.shape)

(768,)


In [18]:
for filename in images:
    image = cv2.imread(filename)
    foreground = np.logical_or(image[:,:,1]<180, image[:,:,0]>150)
    nonzero_x , nonzero_y = np.nonzero(foreground)
    nonzero_cat_values = image[nonzero_x, nonzero_y,:]
    new_frame = background.copy()
    new_frame[nonzero_x,nonzero_y, :] = nonzero_cat_values #Left place
    
        
    image_r=image.copy()
    
    LUT_r = LUT(cdf_image[0:256],cdf_source[0:256])
    image_r[:,:,0] = LUT_r[image_r[:,:,0]]
    
    LUT_g = LUT(cdf_image[256:512],cdf_source[256:512])
    image_r[:,:,1] = LUT_g[image_r[:,:,1]]
    
    LUT_b = LUT(cdf_image[512:768],cdf_source[512:768])
    image_r[:,:,2] = LUT_b[image_r[:,:,2]]
        
    nonzero_cat_values = image_r[nonzero_x, nonzero_y,:]
    new_frame[nonzero_x,background.shape[1]-1-nonzero_y,:]=nonzero_cat_values #right

    new_frame = new_frame[:,:,[2,1,0]] 
    images_list.append(new_frame)


In [19]:
clip = mpy.ImageSequenceClip(images_list, fps=25)
audio = mpy.AudioFileClip('selfcontrol_part.wav').set_duration(clip.duration)
clip = clip.set_audio(audioclip=audio)
clip.write_videofile('part2_video.mp4', codec = 'libx264')


chunk:   0%|                                                                         | 0/159 [00:00<?, ?it/s, now=None]

Moviepy - Building video part2_video.mp4.
MoviePy - Writing audio in part2_videoTEMP_MPY_wvf_snd.mp3


t:   0%|                                                                             | 0/181 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video part2_video.mp4



Moviepy - Done !
Moviepy - video ready part2_video.mp4
